In [ ]:
import telebot
import csv
from datetime import datetime
from telebot.types import ReplyKeyboardMarkup, KeyboardButton, InlineKeyboardMarkup, InlineKeyboardButton

with open("credentials.txt", "r") as file:
    BOT_TOKEN = file.readline().strip().split('=')[1]
    

    
bot = telebot.TeleBot(BOT_TOKEN)

flag = 'neutral'
start = False
inline_keyboard = True

#
#-----------------------------HELPER FUNCTIONS
#
    
    
    
def faq_func(message):
    bot.send_message(message.chat.id, 
                     "Frequently Asked Questions:", 
                     reply_markup=faq_keyboard())
    
    with open("faq.csv", "r") as file:
        # Read the contents of the file using a CSV reader
        csv_reader = csv.DictReader(file)

        # Iterate through the rows in the CSV file
        for row in csv_reader:
            # Extract the question and answer from the row
            question = row["Question"]
            answer = row["Answer"]

            # Send the question and answer as a message
            bot.send_message(message.chat.id, f"{question}")
            bot.send_message(message.chat.id, f"{answer}")

            
def sugg_func(message):
    file_exists = os.path.isfile("suggestions.csv")

    with open("suggestions.csv", "a", newline='') as file:
        # Create a CSV writer
        csv_writer = csv.writer(file)

        # If the file does not exist, write the header row
        if not file_exists:
            header = ['userid', 'username', 'date', 'suggestion']
            csv_writer.writerow(header)

        # Get user_id and current timestamp
        username = message.from_user.username
        user_id = message.from_user.id
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Write the data to the file
        csv_writer.writerow([user_id, username, timestamp, message.text])

    bot.send_message(message.chat.id, "Thank you for submitting your suggestion.", 
                     reply_markup=main_keyboard())


def question_func(message, user_information):
    file_exists = os.path.isfile("questions.csv")
    with open("questions.csv", "a", newline='') as file:
        # Create a CSV writer
        csv_writer = csv.writer(file)
        
        if not file_exists:
            header = ['userid', 'username', 'name', 'email', 'phone', 'date', 'question']
            csv_writer.writerow(header)
            
        # Get current timestamp
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        username = "https://t.me/" + message.from_user.username
        user_id = message.from_user.id
        
        # Write the data to the file
        csv_writer.writerow([user_id,
                            username,
                            *user_information,
                             timestamp,
                             message.text])

    bot.send_message(message.chat.id, 
                     "Your question has been saved. We will get back to you as soon as possible.",
                     reply_markup=main_keyboard())

    
def save_issue_func(message, user_information, issue_title):
    global flag

    file_exists = os.path.isfile("issues.csv")

    with open("issues.csv", "a", newline='') as file:
        # Create a CSV writer
        csv_writer = csv.writer(file)

        if not file_exists:
            header = ['userid', 'username', 'name', 'email', 'phone', 'date', 'title', 'issue']
            csv_writer.writerow(header)

        # Get current timestamp
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        user_id = message.from_user.id
        username = "https://t.me/" + message.from_user.username

        # Write the data to the file
        csv_writer.writerow([user_id,
                             username,
                             *user_information,
                             timestamp,
                             issue_title,
                             message.text])

    bot.send_message(message.chat.id, "Your issue has been reported. We will get back to you as soon as possible.",
                     reply_markup=main_keyboard())
    flag = 'neutral'


    
def options_list(message):
    global inline_keyboard
    if inline_keyboard:
        bot.send_message(message.chat.id,
                     "Please select an option:",
                     reply_markup=main_inline_buttons())


def go_back(message):
    
    global flag
    global user_information
    global issue_title
    
    bot.send_message(message.chat.id, "Main menu:",
                             reply_markup=main_keyboard())
    flag = 'neutral'
    user_information = []
    issue_title = ''

    
    
    
    
    
    
    
#
#----------------------------KEYBOARD BUTTONS
#

def main_keyboard():
    markup = ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=False)
    
    help_button = KeyboardButton("Help")
    markup.add(help_button)
    
    faq_button = KeyboardButton("Frequently Asked Questions")
    markup.add(faq_button)
    
    report_button = KeyboardButton("Report Problem")
    markup.add(report_button)
    
    suggestion_button = KeyboardButton("Suggestions")
    markup.add(suggestion_button)
    
    return markup


def faq_keyboard():
    markup = ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=False)
    
    cant_find_question_button = KeyboardButton("Can't find your answer?")
    markup.add(cant_find_question_button)
    
    go_back_button = KeyboardButton("Go back to main menu")
    markup.add(go_back_button)
    
    return markup


def back_keyboard():
    markup = ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=False)

    go_back_button = KeyboardButton("Go back to main menu")
    markup.add(go_back_button)
    
    return markup


def report_problem_keyboard():
    markup = ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=False)

    box_issue_button = KeyboardButton("Box issue")
    markup.add(box_issue_button)

    invoice_issue_button = KeyboardButton("Invoice issue")
    markup.add(invoice_issue_button)

    others_button = KeyboardButton("Others")
    markup.add(others_button)

    go_back_button = KeyboardButton("Go back to main menu")
    markup.add(go_back_button)

    return markup









#
#----------------------------CALLBACK HANDLERS
#

@bot.callback_query_handler(func=lambda call: call.data == "help")
def handle_help_callback(call):
    print('--> INLINE -> HELP')
    bot.answer_callback_query(call.id)
    help_func(call.message)
    options_list(call.message)

    
    
@bot.callback_query_handler(func=lambda call: call.data == "faq")
def handle_faq_callback(call):
    print('--> INLINE -> FAQ')
    bot.answer_callback_query(call.id)
    faq_func(call.message)
    options_list(call.message)

    

@bot.callback_query_handler(func=lambda call: call.data == "sugg")
def handle_sugg_callback(call):
    print('--> INLINE -> SUGG')
    bot.answer_callback_query(call.id)
    bot.send_message(call.message.chat.id, "We would be glad to hear your voice "
                     "and do our best to grow with you. "
                     "Please let us know what's on your mind.")
    global flag
    flag = 'waiting_for_suggestion'
    options_list(call.message)
    
    
    
    
    
    
    
    
    
#    
#----------------------------INLINE BUTTONS
#

def main_inline_buttons():
    markup = InlineKeyboardMarkup()
    
    help_button = InlineKeyboardButton("Help",
                                       callback_data="help")
    markup.add(help_button)
    
    faq_button = InlineKeyboardButton("Frequently Asked Questions",
                                      callback_data="faq")
    markup.add(faq_button)
    
    suggestion_button = InlineKeyboardButton("Suggestions",
                                             callback_data="sugg")
    markup.add(suggestion_button)
    
    return markup









#
#----------------------------COMMAND HANDLERS
#

@bot.message_handler(commands=['start', 'restart'])
def bot_start(message):
    start = True
    bot.send_message(message.chat.id, """\
GLGMBH Test bot,
Let's cook!
""", reply_markup=main_keyboard())
    
    options_list(message)


@bot.message_handler(commands=['help'])
def help_func(message):
    with open("help.txt", "r") as file:
        help_text = file.read()
    bot.send_message(message.chat.id, help_text)

@bot.message_handler(commands=['toggle_inline_keyboard'])
def toggle_inline_keyboard(message):
    global inline_keyboard
    
    if inline_keyboard:
        inline_keyboard = False
        bot.send_message(message.chat.id, 
                         "Inline keyboard off.")
    else:
        inline_keyboard = True
        bot.send_message(message.chat.id, 
                         "Inline keyboard on.")
        
        
        
        
        
        
        
        
        
#
#----------------------------MESSAGE HANDLER
#

@bot.message_handler(func=lambda message: True)
def bot_operator(message):
    try:
        global flag
        global user_information
        global issue_title

        if flag == 'neutral':
            if message.text.lower() == "help":
                print('--> KEYBOARD -> HELP')
                help_func(message)
                flag = 'neutral'

            elif message.text.lower() == "frequently asked questions":
                print('--> KEYBOARD -> FAQ')
                faq_func(message)
                flag = 'faq'
                
            elif message.text.lower() == "report problem":
                bot.send_message(message.chat.id, "Please select the type of issue you want to report:",
                                 reply_markup=report_problem_keyboard())
                flag = 'waiting_for_issue_type'

            elif message.text.lower() == "suggestions":
                print('--> KEYBOARD -> SUGG')
                bot.send_message(message.chat.id, "We would be glad to hear your voice "
                                 "and do our best to grow with you. "
                                 "Please let us know what's on your mind. ", 
                                reply_markup=back_keyboard())
                flag = 'waiting_for_suggestion'

            else:
                bot.send_message(message.chat.id, "Invalid option. Please try again.")



        elif flag == 'faq':
            if message.text.lower() == "can't find your answer?":
                bot.send_message(message.chat.id, 
                                 "Let us help you with the issue.", 
                                 reply_markup=back_keyboard())

                bot.send_message(message.chat.id, "Please enter your name:")
                flag = 'waiting_for_name_q'

            elif message.text.lower() == "Go back to main menu":
                go_back(message)


        elif flag == 'waiting_for_suggestion':

            if message.text.lower() == "go back to main menu":
                go_back(message)

            else:
                sugg_func(message)
                flag = 'neutral'


        # Handling information for asking questions    
        elif flag == 'waiting_for_name_q':
            if message.text.lower() == "go back to main menu":
                go_back(message)

            else:
                user_information = []
                user_information.append(message.text)
                bot.send_message(message.chat.id, "Please enter your email:")
                flag = 'waiting_for_email_q'

        elif flag == 'waiting_for_email_q':
            if message.text.lower() == "go back to main menu":
                go_back(message)

            else:
                user_information.append(message.text)
                bot.send_message(message.chat.id, "Please enter your phone number:")
                flag = 'waiting_for_phone_q'

        elif flag == 'waiting_for_phone_q':
            if message.text.lower() == "go back to main menu":
                go_back(message)
            else:
                user_information.append(message.text)
                bot.send_message(message.chat.id, "Please enter your question:",
                                 reply_markup=back_keyboard())
                flag = 'waiting_for_question'

        
        elif flag == 'waiting_for_question':
            if message.text.lower() == "go back to main menu":
                go_back(message)
            else:
                question_func(message, user_information)
                flag = 'neutral'
        
        
        
        
        # Handling information for asking issues
        elif flag == 'waiting_for_issue_type':
            if message.text.lower() == "go back to main menu":
                go_back(message)

            elif message.text.lower() in ["box issue", "invoice issue"]:
                issue_title = message.text
                bot.send_message(message.chat.id, "Please enter your name:",
                                reply_markup=back_keyboard())
                flag = 'waiting_for_name_issue'

            elif message.text.lower() in ["others"]:
                bot.send_message(message.chat.id, "Please enter a title for the issue:",
                                reply_markup=back_keyboard())
                flag = 'waiting_for_issue_title'
            
            else:
                bot.send_message(message.chat.id, "Invalid option. Please try again.")
            
        elif flag == 'waiting_for_issue_title':
            if message.text.lower() == "go back to main menu":
                go_back(message)

            else:
                issue_title = message.text
                bot.send_message(message.chat.id, "Please enter your name:",
                            reply_markup=back_keyboard())
            flag = 'waiting_for_name_issue'
                
        elif flag == 'waiting_for_name_issue':
            if message.text.lower() == "go back to main menu":
                go_back(message)

            else:
                user_information = []
                user_information.append(message.text)
                bot.send_message(message.chat.id, "Please enter your email:",
                                reply_markup=back_keyboard())
                flag = 'waiting_for_email_issue'
        
        elif flag == 'waiting_for_email_issue':
            if message.text.lower() == "go back to main menu":
                go_back(message)

            else:
                user_information.append(message.text)
                bot.send_message(message.chat.id, "Please enter your phone number:",
                                reply_markup=back_keyboard())
                flag = 'waiting_for_phone_issue'
                
        elif flag == 'waiting_for_phone_issue':
            if message.text.lower() == "go back to main menu":
                go_back(message)

            else:
                user_information.append(message.text)
                bot.send_message(message.chat.id, "Please give an explanation of your issue:",
                                reply_markup=back_keyboard())
                flag = 'waiting_for_issue_explanation'
                
        elif flag == 'waiting_for_issue_explanation':
            if message.text.lower() == "go back to main menu":
                go_back(message)

            else:
                save_issue_func(message, user_information, issue_title)
                flag = 'neutral'
                    
        
                
        
                
        
        
        
    except Exception as e:
        logging.exception("An error occurred while processing the message:")
        bot.send_message(message.chat.id, "An error occurred. Please restart the bot using /restart.")
        
        
        
        
        
        
        
        
        
#
#----------------------------INFINITE Polling
#

bot.infinity_polling()

--> KEYBOARD -> HELP
--> KEYBOARD -> SUGG
--> KEYBOARD -> FAQ


2023-05-05 18:07:45,834 (__init__.py:960 MainThread) ERROR - TeleBot: "Infinity polling exception: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)"
2023-05-05 18:07:45,836 (__init__.py:962 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/anaconda3/lib/python3.9/http/client.py", line 1377, in getresponse
    response.begin()
  File "/opt/anaconda3/lib/python3.9/http/client.py", line 320, in begin
    version, status, reason = self._read_status()
  File "/opt/anaconda3/lib/python3.9/http/client.py", line 281, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859

In [107]:
user_id = 104764614  # Replace this with the specific user's ID
message_text = "Hello! This is a message from the bot."

bot.send_message(user_id, message_text)
